<a href="https://colab.research.google.com/github/chris051091/Seminario-3/blob/master/preprocess/src/Limpieza%20dato%20de%20estaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpieza dato de estación

Este script realiza la lectura de los datos de eventos que está en formato TXT, hace un pre-procesamiento de los mismos e guarda dentro de una planilla CSV para que sean usados posteriormente dentro de los análisis.

Para una misma estácion, todos los datos serna juntados para criar una única tabela


In [22]:
import pandas as pd
import numpy as np
import os
import re
from functools import reduce
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
PATH_DRIVE = 'drive/MyDrive/Seminario-3-master'
PATH_DADOS = '/preprocess/dados/'

NOMBRE_ESTACION = 'chucuri'
# NOMBRE_ESTACION = 'vizcaina'
# NOMBRE_ESTACION = 'palanquero'

PATH_TIPO_DADO = '/hora/'
FILE_DATOS_PROCESADOS = NOMBRE_ESTACION.lower() + '_procesado.csv'

PATH_FINAL_DADOS = PATH_DRIVE + PATH_DADOS + NOMBRE_ESTACION + PATH_TIPO_DADO

In [23]:
fname = os.listdir(PATH_FINAL_DADOS)

list_df = []

for file in fname:
    print(file)
    name_column_data = re.search(r"^.*_(.*)\.txt", file).group(1)
    with open(f'{PATH_FINAL_DADOS}{file}') as f:
        next(f); next(f) # Saltando dos lineas
        list_data = []
        for line in f:
            dict_data = {}
            if line:
                line_split = line.strip().split()
                dict_data['FECHA'] = ''.join(line_split[0:2])
                dict_data[name_column_data] = line_split[2]
                list_data.append(dict_data)
    
    df = pd.DataFrame(list_data)
    list_df.append(df)
                

PALANQUERO_23035030_TEMP.txt
PALANQUERO_23035030_HUM.txt
PALANQUERO_23035030_PRESION.txt


In [24]:
# Reduciendo dataframes en uno solo
df = reduce(lambda df1,df2: pd.merge(df1,df2,on='FECHA', how='outer'), list_df)

# Formatando data
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%Y%m%d%H:%M:%S').dt.tz_localize('America/Bogota')

In [25]:
df

,FECHA,TEMP,HUM,PRESION
0,2011-01-01 01:00:00-05:00,22.7,99,988.8
1,2011-01-01 02:00:00-05:00,23.1,99,988.4
2,2011-01-01 03:00:00-05:00,23.3,99,988.2
3,2011-01-01 04:00:00-05:00,24.1,95,988.7
4,2011-01-01 05:00:00-05:00,22.9,96,989.4
...,...,...,...,...
48875,2014-02-23 23:44:00-05:00,NaN,NaN,0
48876,2014-02-24 17:48:00-05:00,NaN,NaN,18.7
48877,2014-02-25 11:52:00-05:00,NaN,NaN,0
48878,2014-02-26 05:56:00-05:00,NaN,NaN,0


In [26]:
columns = [col for col in df.columns if col != 'FECHA']

for col in columns:
    # Substituyendo valores no encontrados por np.nan
    df[col] = df[col].replace(['---[10]', '---[2]', '---[5]', '---[15]', '---[3]', '---'], np.NaN)
    # Transformando en formato numérico
    df[col] = pd.to_numeric(df[col], downcast="float")

In [27]:
df.to_csv(PATH_FINAL_DADOS + '../' + FILE_DATOS_PROCESADOS, header=True, index=False, sep = ';')